In [1]:
import great_expectations as ge

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [17]:
spark=SparkSession.builder.appName('ge').getOrCreate()

In [18]:
spark.sparkContext.setLogLevel("WARN")

In [19]:
spark

In [20]:
from pyspark.sql.functions import *

In [21]:
df=spark.read.option('header','true').csv('sales.csv')

In [22]:
df.count()

1000

In [10]:
import datetime
 

from ruamel import yaml
 
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)
 
root_directory = r"C:\Users\hp\Desktop\ge"

In [11]:
from great_expectations import *
#create Data Context
data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)

In [12]:
data_context_config = DataContextConfig(
    validation_operators={
        "action_list_operator": {
            "class_name": "ActionListValidationOperator",
            "action_list": [
                {
                    "name": "store_validation_result",
                    "action": {"class_name": "StoreValidationResultAction"},
                },
                {
                    "name": "store_evaluation_params",
                    "action": {"class_name": "StoreEvaluationParametersAction"},
                },
                {
                    "name": "update_data_docs",
                    "action": {"class_name": "UpdateDataDocsAction"},
                },
            ],
        }
    },
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = BaseDataContext(project_config=data_context_config)

In [13]:
my_spark_datasource_config = {
    "name": "insert_your_datasource_name_here",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "SparkDFExecutionEngine"},
    "data_connectors": {
        "insert_your_data_connector_name_here": {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "some_key_maybe_pipeline_stage",
                "some_other_key_maybe_run_id",
            ],
        }
    },
}
 
context.test_yaml_config(yaml.dump(my_spark_datasource_config))
 
context.add_datasource(**my_spark_datasource_config)
 
batch_request = RuntimeBatchRequest(
    datasource_name="insert_your_datasource_name_here",
    data_connector_name="insert_your_data_connector_name_here",
    data_asset_name="<YOUR_MEANGINGFUL_NAME>",  # This can be anything that identifies this data_asset for you
    batch_identifiers={
        "some_key_maybe_pipeline_stage": "prod",
        "some_other_key_maybe_run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": df},  # Your dataframe goes here
)
 
expectation_suite_name = "insert_your_expectation_suite_name_here"
context.create_expectation_suite(
    expectation_suite_name=expectation_suite_name, overwrite_existing=True
)
# validator = context.get_validator(
#     batch_request=batch_request,
#     expectation_suite_name=expectation_suite_name,
# )

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SparkDFExecutionEngine
Data Connectors:
	insert_your_data_connector_name_here:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



{
  "data_asset_type": null,
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.15.36"
  },
  "ge_cloud_id": null,
  "expectation_suite_name": "insert_your_expectation_suite_name_here"
}

In [14]:
suit_lst = []
col_expectations = {"expect_column_values_to_not_be_null":{"column":"product_code"},"expect_compound_columns_to_be_unique":{"column":"id"},"expect_column_values_to_be_of_type":{"column":"id","datatype":"int"},"expect_column_values_to_be_in_set":{"column":"product_group","value_set":"['PG1','PG2','PG3','PG4', 'PG5']"}}

from great_expectations.core import ExpectationSuite, ExpectationConfiguration
for k,v in col_expectations.items():
    expectation_configuration = ExpectationConfiguration(
    expectation_type =k, kwargs = v
    )
    suit_lst.append(expectation_configuration)

suite = ExpectationSuite("custom_expec_test",expectations=suit_lst)

validator_list = []
validator = context.get_validator(batch_request= batch_request, expectation_suite= suite)

validator.save_expectation_suite(discard_failed_expectations=False)

validator_list.append(validator)
results = context.run_validation_operator("action_list_operator",assets_to_validate=validator_list)

print(results)

Calculating Metrics:   0%|          | 0/19 [00:00<?, ?it/s]

An unexpected Exception occurred during data docs rendering.  Because of this error, certain parts of data docs will not be rendered properly and/or may not appear altogether.  Please use the trace, included in this message, to diagnose and repair the underlying issue.  Detailed information follows:
        TypeError: "object of type 'NoneType' has no len()".  Traceback: "Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\great_expectations\render\renderer\content_block\content_block.py", line 107, in _render_list
    result = content_block_fn(
  File "C:\Users\hp\anaconda3\lib\site-packages\great_expectations\render\renderer\renderer.py", line 14, in inner_func
    return renderer_fn(*args, **kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\great_expectations\expectations\expectation.py", line 118, in inner_func
    ] = render_func(*args, **kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\great_expectations\expectations\core\expect_compoun

{
  "run_id": {
    "run_name": "20221215T102337.503541Z",
    "run_time": "2022-12-15T10:23:37.503541+00:00"
  },
  "success": false,
  "validation_operator_config": {
    "class_name": "ActionListValidationOperator",
    "module_name": "great_expectations.validation_operators",
    "name": "action_list_operator",
    "kwargs": {
      "action_list": [
        {
          "name": "store_validation_result",
          "action": {
            "class_name": "StoreValidationResultAction"
          }
        },
        {
          "name": "store_evaluation_params",
          "action": {
            "class_name": "StoreEvaluationParametersAction"
          }
        },
        {
          "name": "update_data_docs",
          "action": {
            "class_name": "UpdateDataDocsAction"
          }
        }
      ],
      "result_format": {
        "result_format": "SUMMARY",
        "partial_unexpected_count": 20
      }
    }
  },
  "run_results": {
    "ValidationResultIdentifier::custom_

In [34]:
class ge:
    def __init__(self):
        if results:
            eval_rows = df.count()
            validation_ids = [res for res in results['run_results']]
            validations_results_dict = results.to_json_dict()
            if validation_ids != []:
                bad_rec_count_ret =self._get_bad_records(validation_ids= validation_ids, validations_results_dict=validations_results_dict, df=data_df)
                if bad_rec_count_ret== None: 
                    bad_rec_count_ret==0
            result_format = validations_results_dict['validation_operator_config']['kwargs']['result_format']['result_format']
            results_object = []
            resultss = validations_results_dict['run_results'][str(validation_ids[0])]['validation_result']
            val_time = resultss['meta']['validation_time']
            match = re.search(r'\d{4}\d{2}\d{2}', val_time)
            val_date = dt.strptime(match.group(), '%Y%m%d').date() 
            validation_time = val_date.strftime("%Y-%m-%d")
            file_name = resultss['results'][0]['expectation_config']['meta']['file_name']
            success = resultss['success']
            for f in resultss['results']:
                if f['expectation_config']['expectation_type'] == "expect_column_value_to_be_in_normal_range":
                    print("inn")
                    f.update({'validation_time' : validation_time})
                    results_object.append(f)
                else:
                    f.update({'result' : {'element_count' : f['result'].get('element_count'),'overall_success':success,'result_format':result_format, 'unexpected_count' : f['result'].get('unexpected_count')},'validation_time' : validation_time})
                    results_object.append(f)
        resultss['statistics'].update({'success':success, 'validation_time' : validation_time})
        resultss['statistics'].update({'unexpected_rows': bad_rec_count_ret,'evaluated_rows':eval_rows,'file_name':file_name})
        stat = {'statistics' : resultss['statistics'] }
        results_object.append(stat)
        if len(validation_ids) > 1:
            stats = validations_results_dict['run_results'][str(validation_ids[1])]['validation_result']
            stats['statistics'].update({'success':stats['success'], 'validation_time' : validation_time,'file_name':file_name})
            stat_pandas = {'statistics' : stats['statistics']}
            results_object.append(stat_pandas)
            result_path = "C:/Users/hp/Desktop"
            file = f'{str(result_path)}/result_{str(file_date)}.json'
            os.makedirs(os.path.dirname(file), exist_ok=True)
        

                
        
        


In [29]:
ge

__main__.ge

In [30]:
results

{
  "run_id": {
    "run_name": "20221215T102337.503541Z",
    "run_time": "2022-12-15T10:23:37.503541+00:00"
  },
  "success": false,
  "validation_operator_config": {
    "class_name": "ActionListValidationOperator",
    "module_name": "great_expectations.validation_operators",
    "name": "action_list_operator",
    "kwargs": {
      "action_list": [
        {
          "name": "store_validation_result",
          "action": {
            "class_name": "StoreValidationResultAction"
          }
        },
        {
          "name": "store_evaluation_params",
          "action": {
            "class_name": "StoreEvaluationParametersAction"
          }
        },
        {
          "name": "update_data_docs",
          "action": {
            "class_name": "UpdateDataDocsAction"
          }
        }
      ],
      "result_format": {
        "result_format": "SUMMARY",
        "partial_unexpected_count": 20
      }
    }
  },
  "run_results": {
    "ValidationResultIdentifier::custom_